In [ ]:
%pip install mlflow pandas sklearn

In [8]:
import getpass
import os

os.environ["MLFLOW_TRACKING_URI"] = "https://mlflow.spencerlyon.com"
os.environ["MLFLOW_EXPERIMENT_NAME"] = "ucf-kaggle-titanic-spaceship"
os.environ["MLFLOW_TRACKING_USERNAME"] = "TODO"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "TODO"

import pandas as pd
import sklearn
import mlflow
mlflow.autolog()

import mlflow.sklearn
from sklearn import linear_model, model_selection, metrics

if "MLFLOW_TRACKING_PASSWORD" not in os.environ:
    raise ValueError("You need to properly configure the `.env` file")
else:
    print("good work! mlflow ready to go")

mlflow.set_experiment(os.environ.get("MLFLOW_EXPERIMENT_NAME", "Default"))

2022/10/28 19:18:14 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/10/28 19:18:14 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


good work! mlflow ready to go


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1666987017684, experiment_id='1', last_update_time=1666987017684, lifecycle_stage='active', name='ucf-kaggle-titanic-spaceship', tags={}>

In [9]:
df = pd.read_csv("https://raw.githubusercontent.com/sglyon/UCF-MSDA-workshop/master/Year21-22/2022-10-28_kaggle_titantic_spaceship/train.csv")
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [ ]:
feature_cols = ["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]
target_col = "Transported"

no_na = df.dropna()

X = no_na[feature_cols]
y = no_na[target_col]

In [ ]:
train_X, test_X, train_y, test_y = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
with mlflow.start_run(tags=dict(who=getpass.getuser())):
    model = linear_model.LogisticRegression()
    model.fit(train_X, train_y)
    mlflow.sklearn.eval_and_log_metrics(model, test_X, test_y, prefix="val_")